In [ ]:
# Install required packages
!pip install transformers torch langchain faiss-cpu sentence-transformers numpy
!pip install -U langchain-community
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 58.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
# Import libraries
from google.colab import drive
import torch
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForCausalLM # Make sure this line is present
import faiss

# Mount Google Drive
# drive.mount('/content/drive')

# Paths to your files in Google Drive
model_path = "/content/drive/MyDrive/phi_finetuned1"  # Update

srs_file_path = "doc.txt"  # Update if needed

print("Setup Complete!")

Setup Complete!


In [ ]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Load embedding model for RAG
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Function to create FAISS vector store from SRS document
def create_vector_store(srs_text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)
    chunks = text_splitter.split_text(srs_text)
    vectors = embedding_model.embed_documents(chunks)
    vectors = np.array(vectors, dtype=np.float32)
    index = faiss.IndexFlatL2(vectors.shape[1])
    index.add(vectors)
    faiss.write_index(index, "srs_faiss.index")
    return index, chunks

# Load SRS Document
with open(srs_file_path, "r", encoding="utf-8") as file:
    srs_text = file.read()

# Create FAISS vector store
index, chunks = create_vector_store(srs_text)
print("SRS Document Indexed for RAG!")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<ipython-input-3-40c39fe4fb8e>:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SRS Document Indexed for RAG!


In [ ]:
# Function to retrieve relevant chunks
def retrieve_relevant_chunks(query, top_k=3):
    query_embedding = np.array(embedding_model.embed_query(query)).reshape(1, -1)
    index = faiss.read_index("srs_faiss.index")
    distances, indices = index.search(query_embedding, top_k)
    retrieved_docs = [chunks[i] for i in indices[0]]
    return " ".join(retrieved_docs)

# Function to generate test cases
def generate_test_cases(srs_query, max_length=1000):
    context = retrieve_relevant_chunks(srs_query)
    prompt = f"Generate test cases based on the following with detail explanation:\n{context}\nTest Cases:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    output = model.generate(**inputs, max_length=max_length, do_sample=True, top_k=50, temperature=0.7)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    # Extract test cases after "Test Cases:"
    test_cases_start = response.find("Test Cases:")
    if test_cases_start != -1:
        test_cases = response[test_cases_start + len("Test Cases:"):].strip()
        return test_cases, prompt  # Return prompt for RL later
    return response, prompt  # Fallback

# Example Query
query = "Generate test cases for Functional Requirements"
test_cases, prompt = generate_test_cases(query)
print("Initial Test Cases:\n", test_cases)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Initial Test Cases:
 Test Case 1: User Registration

1. Open the Online Shopping System
2. Enter a valid email and password
3. Click on Create Account
4. Verify that a confirmation email is sent to the email address
5. Verify that the confirmation link is displayed on the screen
6. Click on the confirmation link
7. Verify that a message is displayed saying that the email has been verified
8. Verify that a new account page is displayed

Expected Result: All steps are completed successfully and the user is registered with a valid email and password.

Test Case 2: User Login

1. Open the Online Shopping System
2. Enter a valid email and password
3. Click on Login
4. Verify that the user is redirected to the dashboard page
5. Verify that the user name is displayed on the top right corner of the page
6. Verify that the logout button is visible on the top right corner of the page

Expected Result: All steps are completed successfully and the user is authenticated and logged in.
